In [53]:
import pandas as pd
import math
filename = 'Beautiful adhkar.csv'
df = pd.read_csv(filename)
# df.head()

In [54]:
df.head()

,ID,arabic,english,secondsToRecite,minutesToRecite,shortDescription
0,1,NaN,NaN,120,2.0,salawat x 33
1,2,NaN,NaN,360,6.0,salawat x 100
2,3,NaN,NaN,60,1.0,"subhanallah x 33, alhamdulillah x 33, allahuak..."
3,4,NaN,NaN,18,0.3,"subhanallah x 10, alhamdulillah x 10, allahuak..."
4,5,NaN,NaN,0,NaN,hasbunallahu wa nimal wakeel


In [73]:
import sqlite3
import os

# Create a SQL connection to our SQLite database
con = sqlite3.connect(os.path.join("..","dataentry.sqlite3"))

cur = con.cursor()

# The result of a "cursor.execute" can be iterated over by row
for row in cur.execute('SELECT * FROM adhkar;'):
    print(row)


(1, None, None, 120, 2, 'salawat x 33')
(2, None, None, 360, 6, 'salawat x 100')
(3, None, None, 60, 1, 'subhanallah x 33, alhamdulillah x 33, allahuakbar x 33')
(4, None, None, 18, 0, 'subhanallah x 10, alhamdulillah x 10, allahuakbar x 10')
(5, None, None, 0, 0, 'hasbunallahu wa nimal wakeel')
(6, None, None, 18, 0, 'la ilaha illallah x 10')
(7, None, None, 30, 0, 'ayat al kursi')
(8, None, None, 30, 0, 'al fatihah')
(9, None, None, 6, 0, 'Q2:201')
(10, None, None, 60, 1, 'astaghfirullah x 100')
(11, None, None, 30, 0, 'la hawla wa la quwwata illa billah x 10')
(12, None, None, 90, 1, 'la hawla wa la quwwata illa billah x 33')
(13, None, None, 300, 5, 'la hawla wa la quwwata illa billah x 100')


In [74]:
for index, row in df.iterrows():
    shortDesc = row['shortDescription']
    search = (shortDesc,) #https://docs.python.org/2/library/sqlite3.html
    cur.execute('SELECT * FROM adhkar WHERE shortDescription=?', search)
    db_result = cur.fetchone()
    if(db_result):
        print('already_found:',db_result)
    else:
        print('not found:',search)
        seconds = row['secondsToRecite']
        minutes = row['minutesToRecite']
        
        if(math.isnan(minutes) or math.isnan(seconds)):
            if(math.isnan(minutes) and math.isnan(seconds)):
                continue;
            if(math.isnan(minutes)):
                minutes = seconds / 60
            if(math.isnan(seconds)):
                seconds = minute * 60

                
        seconds = int(seconds)
        minutes = int(minutes)
        
        adhkar_row = (row['arabic'], row['english'], seconds, minutes, row['shortDescription'])
        
        cur.execute("INSERT INTO adhkar (arabic, english, secondsToRecite, minutesToRecite, shortDescription) VALUES (?,?,?,?,?)", adhkar_row)
        # could batch these up and do them at the end 
        # but if we do it per-row it avoids duplicates in the same file. 
        # so it's better todo it per-row.
        
for row in cur.execute('SELECT * FROM adhkar;'):
    print(row)



already_found: (1, None, None, 120, 2, 'salawat x 33')
already_found: (2, None, None, 360, 6, 'salawat x 100')
already_found: (3, None, None, 60, 1, 'subhanallah x 33, alhamdulillah x 33, allahuakbar x 33')
already_found: (4, None, None, 18, 0, 'subhanallah x 10, alhamdulillah x 10, allahuakbar x 10')
already_found: (5, None, None, 0, 0, 'hasbunallahu wa nimal wakeel')
already_found: (6, None, None, 18, 0, 'la ilaha illallah x 10')
already_found: (7, None, None, 30, 0, 'ayat al kursi')
already_found: (8, None, None, 30, 0, 'al fatihah')
already_found: (9, None, None, 6, 0, 'Q2:201')
already_found: (10, None, None, 60, 1, 'astaghfirullah x 100')
already_found: (11, None, None, 30, 0, 'la hawla wa la quwwata illa billah x 10')
already_found: (12, None, None, 90, 1, 'la hawla wa la quwwata illa billah x 33')
already_found: (13, None, None, 300, 5, 'la hawla wa la quwwata illa billah x 100')
(1, None, None, 120, 2, 'salawat x 33')
(2, None, None, 360, 6, 'salawat x 100')
(3, None, None, 60

In [71]:
con.commit()
con.close()